# systemd

## Overview:
Services are processes that run in the background of a system, out of the user’s view. They typically lack an interface and therefore have to be interacted with using commands. Services streamline the user experience by taking care of operations automatically and in a way that does not distract the user. There is actually a service that manages the starting and stopping of other services, systemd. This lab provides the basics of starting and stopping services using systemctl, which is the command that allows users to interact with the systemd service.

## Goal:
After completing this lab, users will understand how to start, restart, and stop a service. You will also learn how to make a service start every time the system starts.

- Concepts included in this scenario:
- View and kill processes
- Learn how processes and services are related
- View the status of a service
- Learn how systemd and systemctl are related
- Enable a service with systemctl to make it persistent
- Restart a service to activate configuration changes

## Step 1
What are processes?
Files keep track of information on a Linux system, providing a way for users to organize and store information. Sometimes, files contain information that can be run to carry out a specific operation. These kinds of files are called programs. When you run a program, the running instance is called a process. Modern Linux systems have tons of processes running at any given moment, so it is crucial to have a way to manage these processes.

Viewing processes
The command that lets you view processes on Linux is ps, short for "Process Status". This command will show you all processes running in the current shell if called without any options. However, it is frequently useful to see processes that may not have been started with your user ID. The command ps aux will show you all processes, including those running without a controlling terminal. This command will also give information about the users associated with these processes.

```bash
ps aux
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.7 186268 14440 ?        Ss   16:09   0:03 /usr/lib/systemd/systemd --switched-root --system --deserialize 17
root           2  0.0  0.0      0     0 ?        S    16:09   0:00 [kthreadd]
root           3  0.0  0.0      0     0 ?        I<   16:09   0:00 [rcu_gp]
root           4  0.0  0.0      0     0 ?        I<   16:09   0:00 [rcu_par_gp]

<< OUTPUT ABRIDGED >>
```

This output is very extensive, so often when you call this command you want to pipe the output into grep to search for a specific phrase. To practice this, start a process in Terminal 2 and then search for it in the original terminal. The dd command can be used to create an indefinite process in Terminal 2. The command below begins copying data from a file of infinite zeros into a file that discards all inputs, so the process will continue indefinitely.

```bash
dd if=/dev/zero of=/dev/null
```

When you run this command, you will not see an output as the process will continue running in the foreground until you interrupt it. Returning to the original terminal, run a search on ps aux to find this indefinite dd process:

```bash
ps aux | head -n1 ; ps aux | grep '[d]d '
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root        3790  101  0.0   7352   944 pts/1    R+   17:38   0:01 dd if=/dev/zero of=/dev/null
```

NOTE: The first command in this line will print the column headers for ps aux so that you can see what each entry means. The second command searches for a process called dd , where the single quotes ensure the trailing space is part of the search criteria. Enclosing the first "d" in square brackets prevents the grep search from finding itself when it looks through the process list.

Take note of the second column, the Process ID. This ID is how you interact with the process, as it can be used in other commands as a unique reference to this process. Now you know how to find a process. The next step will walk you through terminating this indefinite process.

## Step 2
Killing processes
Now that you found the dd process on Terminal 1, terminate it using the kill command. You would typically just manually enter the process ID after kill. The process ID is the second column of the ps aux output from before. However, to make this command clickable for each new lab instance the pidof (__P__rocess ID of) command is used to automatically find the process ID.

```bash
kill $(pidof dd)
There is no output, but switching back to Terminal 2 reveals that the dd process has been terminated.
```

## Terminated
The kill command can send a variety of signals. Calling the command without any options will default to -SIGTERM. This command will inform the process that it is time for it to stop, but allow it to run any cleanup procedures that it has. This often takes the form of closing files and freeing memory. In this sense, -SIGTERM is the graceful option for terminating a process.

If you instead use the kill -SIGKILL command, this will instruct the kernel to immediately stop the process. This prevents any cleanup that may have otherwise occurred, leaving memory allocated and potentially leading to corrupted files. Therefore, only use -SIGKILL as a last resort.

NOTE: You will also see the kill signals referred to numerically. -SIGTERM

is equivalent to -15 and -SIGKILL is equivalent to -9.

## Step 3
What is a service?
The process you interacted with in the previous step was a foreground process. It ran in a terminal window where you could see it. Some processes run in the background, out of view of the user. These background processes allow the system to carry out many operations in parallel. Background processes that continuously carry out a set of actions are called services.

Daemons are even more specialized. These are services that and are specifically designed to supervise or support other processes. Typically, daemons are denoted by a d at the end of their name. For example, firewalld is the daemon which handles firewall functionality. These daemons lie in wait, listening for the user to issue a command to tell them to change their behavior.

Viewing the status of a service
firewalld is a service which manages what network traffic to let into the system. Check the status of the firewalld service with the following command:

```bash
systemctl status firewalld.service --no-pager
● firewalld.service - firewalld - dynamic firewall daemon
   Loaded: loaded (/usr/lib/systemd/system/firewalld.service; enabled; vendor preset: enabled)
   Active: active (running) since Mon 2021-06-28 15:50:21 EDT; 1h 49min ago
     Docs: man:firewalld(1)
 Main PID: 875 (firewalld)
    Tasks: 2 (limit: 11380)
   Memory: 35.3M

<< OUTPUT ABRIDGED >>
```

From this status message it is clear that the firewalld service is installed and active. But what is managing this service? It turns out that another service, systemd, is in charge of managing all of the services on the system.

systemd vs. systemctl
When researching service management, you will come across two very similar terms: systemd and systemctl. These are very closely related. systemd, short for system daemon, manages the state of the system and any services running on it. Since systemd is a daemon, it runs in the background and needs a set of commands for users to interact with it. systemctl provides these commands. The systemctl status command above is one example of this, and the upcoming steps will walk you through starting and enabling a service with systemctl.

## Step 4
Starting and enabling a service with systemctl
For this example, you will be using the Network File System (NFS) service. This service allows users on client systems to view files and directories over a network as if they were on the client's local drive. Rather than focusing on the functionality of this service, though, this lab simply uses it as an example of how to start a service and implement configuration file changes.

To begin, run the command to show the export list for the NFS server:

```bash
showmount -e
```

This throws an error, because the NFS service is not running.

clnt_create: RPC: Program not registered
To solve this issue, you need to start the service. However, if you want the service to start each time you boot the system, you need to enable the service. You can do both of these operations in one line:

```bash
systemctl enable --now nfs-server
Created symlink /etc/systemd/system/multi-user.target.wants/nfs-server.service → /usr/lib/systemd/system/nfs-server.service
```

Note: If you just wanted to start a service, systemctl start

followed by the service name will do the trick. However, the service will not start again if you reboot the system.

Now the nfs-server service is started, and it will start every time the system starts as well. Run the showmount command again now that the service is active:

```bash
showmount -e

Export list for rhel:
```

The export list is currently blank, which is something controlled by a configuration file. You may assume that changing a config file would immediately lead to changes in the service, but this is not the case. The next step will walk you through restarting a service to reload configuration files.

## Step 5
Restarting a service to apply configuration changes
To demonstrate how configuration changes behave when a service is already running, add an export directory to the NFS service on this system:

```bash
echo "/home" >> /etc/exports
```

This command adds the home directory to the export list configuration file for NFS. Changing this config file does not lead to any immediate changes in the NFS service, though. Print the export list again to confirm this:

```bash
showmount -e
Export list for rhel:
```

To bring configuration changes into the service, you must first use systemctl to restart it:

```bash
systemctl restart nfs-server
```

Now the configuration files have been re-read and NFS reflects the configuration change.

```bash
showmount -e
Export list for rhel:
/home *
```

The home directory is now listed as an exported drive that is available to all NFS clients. If you ever run into a scenario where you are unable to see configuration changes, make sure you check whether or not you have restarted the service and reloaded the config files.

<p style="text-align: center">
  <img  src="Media/systemd.png" width="600" alt="systemd">
</p>

